In [20]:
from layers import conv_forward_naive

def conv_backward_naive(dout, cache):
  """
  A naive implementation of the backward pass for a convolutional layer.

  Inputs:
  - dout: Upstream derivatives.
  - cache: A tuple of (x, w, b, conv_param) as in conv_forward_naive

  Returns a tuple of:
  - dx: Gradient with respect to x
  - dw: Gradient with respect to w
  - db: Gradient with respect to b
  """
  print "In"
  dx, dw, db = None, None, None
  #############################################################################
  # TODO: Implement the convolutional backward pass.                          #
  #############################################################################
  x, w, b, conv_param = cache
  pad = conv_param['pad'] 
  stride = conv_param['stride']
  
  N, F, H_1, W_1 = dout.shape
  F, C, HH, WW = w.shape
  N, C, H, W = x.shape
  
  inv_pad = ((W - 1) * stride + WW - W_1) / 2
  dout_pad = np.zeros((N, F, H_1+2*inv_pad, W_1+2*inv_pad))
  # padding dout
  for n in xrange(N):
    for f in xrange(F):
      dout_pad[n,f,:,:] = np.pad(dout[n,f,:,:], inv_pad, 'constant')

  dx = np.zeros_like(x)
  # convolution, naive implementation
  for n in xrange(N):
      ii = 0
      for i in xrange(H):
          jj = 0
          for j in xrange(W):
              for c in xrange(C):
                  for f in xrange(F):
                      dx[n, c, i, j] += np.sum(dout_pad[n, f, ii:ii+HH, jj:jj+WW] * w[f, c, :, :])
                      print "iteration.."
                      print dx[n, c, :, :]
              jj += stride
                  # print "ii=%s, jj=%s"% (ii, jj)
          ii += stride
                                 
  
  #############################################################################
  #                             END OF YOUR CODE                              #
  #############################################################################
  return dx, dw, db


x = np.random.randint(0,5,(4, 3, 3, 3))
w = np.random.randint(0,5,(2, 3, 2, 2))
b = np.random.randint(0,5,(2,))
dout = np.random.randint(0,5,(4, 2, 2, 2))
conv_param = {'stride': 1, 'pad': 0}


out, cache = conv_forward_naive(x, w, b, conv_param)
print "dout"
print dout[0,0,:,:]
print "weight"
print w[:,0,:,:]
dx, dw, db = conv_backward_naive(dout, cache)


dout
[[4 1]
 [3 3]]
weight
[[[1 2]
  [3 0]]

 [[0 3]
  [2 1]]]
In
iteration..
[[0 0 0]
 [0 0 0]
 [0 0 0]]
iteration..
[[2 0 0]
 [0 0 0]
 [0 0 0]]
iteration..
[[16  0  0]
 [ 0  0  0]
 [ 0  0  0]]
iteration..
[[22  0  0]
 [ 0  0  0]
 [ 0  0  0]]
iteration..
[[12  0  0]
 [ 0  0  0]
 [ 0  0  0]]
iteration..
[[16  0  0]
 [ 0  0  0]
 [ 0  0  0]]
iteration..
[[ 2 12  0]
 [ 0  0  0]
 [ 0  0  0]]
iteration..
[[ 2 16  0]
 [ 0  0  0]
 [ 0  0  0]]
iteration..
[[22  4  0]
 [ 0  0  0]
 [ 0  0  0]]
iteration..
[[22  4  0]
 [ 0  0  0]
 [ 0  0  0]]
iteration..
[[16  3  0]
 [ 0  0  0]
 [ 0  0  0]]
iteration..
[[16  7  0]
 [ 0  0  0]
 [ 0  0  0]]
iteration..
[[ 2 16  3]
 [ 0  0  0]
 [ 0  0  0]]
iteration..
[[ 2 16  3]
 [ 0  0  0]
 [ 0  0  0]]
iteration..
[[22  4  0]
 [ 0  0  0]
 [ 0  0  0]]
iteration..
[[22  4  0]
 [ 0  0  0]
 [ 0  0  0]]
iteration..
[[16  7  0]
 [ 0  0  0]
 [ 0  0  0]]
iteration..
[[16  7  0]
 [ 0  0  0]
 [ 0  0  0]]
iteration..
[[ 2 16  3]
 [ 8  0  0]
 [ 0  0  0]]
iteration..
[[ 2 16  

In [ ]:
import numpy as np

from cs231n.layers import *
from cs231n.fast_layers import *
from cs231n.layer_utils import *


class ThreeLayerConvNet(object):
  """
  A three-layer convolutional network with the following architecture:
  
  conv - relu - 2x2 max pool - affine - relu - affine - softmax
  
  The network operates on minibatches of data that have shape (N, C, H, W)
  consisting of N images, each with height H and width W and with C input
  channels.
  """
  
  def __init__(self, input_dim=(3, 32, 32), num_filters=32, filter_size=7,
               hidden_dim=100, num_classes=10, weight_scale=1e-3, reg=0.0,
               dtype=np.float32):
    """
    Initialize a new network.
    
    Inputs:
    - input_dim: Tuple (C, H, W) giving size of input data
    - num_filters: Number of filters to use in the convolutional layer
    - filter_size: Size of filters to use in the convolutional layer
    - hidden_dim: Number of units to use in the fully-connected hidden layer
    - num_classes: Number of scores to produce from the final affine layer.
    - weight_scale: Scalar giving standard deviation for random initialization
      of weights.
    - reg: Scalar giving L2 regularization strength
    - dtype: numpy datatype to use for computation.
    """
    self.params = {}
    self.reg = reg
    self.dtype = dtype
    
    ############################################################################
    # TODO: Initialize weights and biases for the three-layer convolutional    #
    # network. Weights should be initialized from a Gaussian with standard     #
    # deviation equal to weight_scale; biases should be initialized to zero.   #
    # All weights and biases should be stored in the dictionary self.params.   #
    # Store weights and biases for the convolutional layer using the keys 'W1' #
    # and 'b1'; use keys 'W2' and 'b2' for the weights and biases of the       #
    # hidden affine layer, and keys 'W3' and 'b3' for the weights and biases   #
    # of the output affine layer.                                              #
    ############################################################################
    C, H, W = input_dim
    HH, WW = filter_size, filter_size
    F = num_filters
    
    self.params['W1'] = weight_scale * np.random.randn(F,C,HH,WW)
    self.params['b1'] = np.zeros(F)
    affine1_dim = (H/2) * (W/2) * (num_filters)
    self.params['W2'] = weight_scale * np.random.randn(affine1_dim, hidden_dim)
    self.params['b2'] = np.zeros(hidden_dim)
    self.params['W3'] = weight_scale * np.random.randn(hidden_dim, num_classes)
    self.params['b3'] = np.zeros(num_classes)
    ############################################################################
    #                             END OF YOUR CODE                             #
    ############################################################################

    for k, v in self.params.iteritems():
      self.params[k] = v.astype(dtype)
     
 
  def loss(self, X, y=None):
    """
    Evaluate loss and gradient for the three-layer convolutional network.
    
    Input / output: Same API as TwoLayerNet in fc_net.py.
    """
    W1, b1 = self.params['W1'], self.params['b1']
    W2, b2 = self.params['W2'], self.params['b2']
    W3, b3 = self.params['W3'], self.params['b3']
    
    # pass conv_param to the forward pass for the convolutional layer
    filter_size = W1.shape[2]
    conv_param = {'stride': 1, 'pad': (filter_size - 1) / 2}

    # pass pool_param to the forward pass for the max-pooling layer
    pool_param = {'pool_height': 2, 'pool_width': 2, 'stride': 2}

    scores = None
    ############################################################################
    # TODO: Implement the forward pass for the three-layer convolutional net,  #
    # computing the class scores for X and storing them in the scores          #
    # variable.                                                                #
    ############################################################################
    # forward pass
    layer1, cache1 = conv_relu_pool_forward(X, W1, b1, conv_param, pool_param)
    layer2, cache2 = affine_relu_forward(layer1, W2, b2)
    layer3, cache3 = affine_forward(layer2, W3, b3)
    scores = layer3
    ############################################################################
    #                             END OF YOUR CODE                             #
    ############################################################################
    
    if y is None:
      return scores
    
    loss, grads = 0, {}
    ############################################################################
    # TODO: Implement the backward pass for the three-layer convolutional net, #
    # storing the loss and gradients in the loss and grads variables. Compute  #
    # data loss using softmax, and make sure that grads[k] holds the gradients #
    # for self.params[k]. Don't forget to add L2 regularization!               #
    ############################################################################
    loss, dloss = softmax_loss(scores, y)
    reg = self.reg
    loss += 0.5 * reg * (np.sum(W1*W1) + np.sum(W2*W2) + np.sum(W3*W3))
    #backward pass
    dlayer3, dW3, db3 = affine_backward(dloss, cache3)
    dlayer2, dW2, db2 = affine_relu_backward(dlayer3, cache2)
    dx, dW1, db1 = conv_relu_pool_backward(dlayer2, cache1)
    
    
    grads['W1'] = dW1 + reg * W1
    grads['b1'] = db1
    grads['W2'] = dW2 + reg * W2
    grads['b2'] = db2
    grads['W3'] = dW3 + reg * W3
    grads['b3'] = db3
    ############################################################################
    #                             END OF YOUR CODE                             #
    ############################################################################
    
    return loss, grads
  
  
pass
